In [24]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import string
import pickle as pkl

In [25]:
print(tf.__version__)
print(keras.__version__)

2.0.0-alpha0
2.2.4-tf


In [30]:
def preprocess(filepath):
    """To be completed"""
    with open(filepath) as f:
        text = f.read()
    # Remove punctuation from each word
    punct = str.maketrans("", "", string.punctuation)
    text = [w.translate(punct) for w in text]
    print(text[:100])
    
    # Create mappings characters - integers
    # Converts to lowercase by default
    tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
    tokenizer.fit_on_texts([text])
        
    # Prepare dataset of input to output pairs encoded as integers
    max_id = len(tokenizer.word_index)
    print(max_id)
    dataset_size = tokenizer.document_count
    print(dataset_size)
    [encoded] = np.array(tokenizer.texts_to_sequences([text])) - 1
    print(encoded, len(encoded))
    train_size = len(encoded) * 90 // 100
    dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
    n_steps = 100
    window_length = n_steps + 1
    dataset = dataset.window(window_length, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    batch_size = 32
    dataset = dataset.shuffle(10000).batch(batch_size)
    dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
    dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
    dataset = dataset.prefetch(1)
    
    np.save("/home/tdelatte/Projects/Deep_Learning/NLP/Baudelaire_Poem_Generator/data/raw/dataset.npy", dataset)

In [31]:
preprocess("/home/tdelatte/Projects/Deep_Learning/NLP/Baudelaire_Poem_Generator/data/raw/baudelaire.txt")

['À', ' ', 'c', 'e', 'l', 'l', 'e', ' ', 'q', 'u', 'i', ' ', 'e', 's', 't', ' ', 't', 'r', 'o', 'p', ' ', 'g', 'a', 'i', 'e', '\n', 'T', 'a', ' ', 't', 'ê', 't', 'e', '', ' ', 't', 'o', 'n', ' ', 'g', 'e', 's', 't', 'e', '', ' ', 't', 'o', 'n', ' ', 'a', 'i', 'r', ' ', '\n', 'S', 'o', 'n', 't', ' ', 'b', 'e', 'a', 'u', 'x', ' ', 'c', 'o', 'm', 'm', 'e', ' ', 'u', 'n', ' ', 'b', 'e', 'a', 'u', ' ', 'p', 'a', 'y', 's', 'a', 'g', 'e', ' ', '', ' ', '\n', 'L', 'e', ' ', 'r', 'i', 'r', 'e', ' ', 'j']
56
1
[27  0 15 ...  0 11 14] 235437


RuntimeError: Tensorflow type 21 not convertible to numpy dtype.